# Práctica 5 Automatización y Optimización Avanzada

> Objetivo central: **"Automatiza un stack CRUD completo (modelo, API, tests e infraestructura) orquestando LLMs con LangChain de forma reproducible y medible."**

## ¿Qué voy a lograr y por qué importa?
En esta práctica construyes un **pipeline automatizado** que, partiendo de una configuración declarativa (YAML / Pydantic), genera:
- Modelos Pydantic validados
- Router FastAPI empresarial (CRUD + paginación + auth opcional)
- Suite de tests Pytest
- Infraestructura (Dockerfile, migración Alembic)
- Métricas de eficiencia de la generación

Esto refleja un caso real: equipos que necesitan **acelerar el scaffolding backend** manteniendo estándares de calidad. Aprenderás a usar **LangChain Expression Language (LCEL)** y `RunnableParallel` para ejecutar tareas en paralelo y encadenar dependencias.

Problemas reales que esto resuelve:
- Onboarding lento: crear cada CRUD manualmente tarda horas.
- Inconsistencia entre servicios: estilos diferentes de validación / logs.
- Falta de medición: se generan cosas con IA pero sin métricas.
- Riesgo técnico: prompts desordenados, sin control de dependencias.

Solución mostrada: un **pipeline determinista** donde cada bloque tiene una responsabilidad clara. Así escalas generación de servicios sin sacrificar mantenibilidad.

Rol de las piezas:
- `ResourceConfig` y `FieldConfig`: contrato declarativo de tu recurso.
- Prompts especializados (model, router, tests, infra): separación de dominios (Domain Prompting).
- `RunnableParallel`: acelera la generación base (modelo + config) y luego deriva dependientes.
- `Structured Output`: fuerza esquemas (`InfrastructureComponents`).
- Métricas: base para gobernanza y ROI de IA.

| Concepto | Idea-faro | Analogía |
|----------|-----------|----------|
| LCEL | Orquesta modular | "LEGO de flujos LLM" |
| `RunnableParallel` | Paralelismo declarativo | "Carriles simultáneos" |
| Config → Artefactos | Infra como código pero para scaffolding | "Terraform de tu backend" |
| Prompts especializados | Principio de responsabilidad única | "Microservicios cognitivos" |
| Métricas | Observabilidad del pipeline | "Tablero de control DevOps" |
| Structured Output | Control sintáctico | "Molde para la arcilla del modelo" |


## Práctica paso a paso

### Parte 1: Setup del Entorno

Configuraremos un entorno completo con herramientas de análisis y generación automatizada.

In [ ]:
!pip install langchain langchain-openai langchain-community fastapi uvicorn pydantic pytest httpx python-dotenv jinja2 pyyaml click rich

In [ ]:
import os
import yaml
import json
import ast
import time
import logging
from pathlib import Path
from typing import Dict, List, Optional, Any, Literal
from dataclasses import dataclass
from datetime import datetime
from jinja2 import Template

# LangChain imports
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser, StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnableLambda
from langchain_core.callbacks import BaseCallbackHandler
from pydantic import BaseModel, Field


In [ ]:

from dotenv import load_dotenv
load_dotenv()

if not os.getenv("OPENAI_API_KEY"):
    print("⚠️  Configura OPENAI_API_KEY en tu archivo .env")
else:
    print("✅ OpenAI API Key configurada")

# Configurar modelo
model = ChatOpenAI(model="gpt-5", temperature=0)
print("🤖 Modelo listo")

### 4.2 Modelos de Configuración
Implementaremos un sistema avanzado de generación CRUD usando RunnableParallel y configuración YAML.

### Parte 3: Generadores Especializados con RunnableParallel

Crearemos generadores especializados que trabajen en paralelo para máxima eficiencia.

Se crean 4 generadores:
1. `model_generator`: produce modelos Pydantic (entrada, salida, update, validaciones y validators).
2. `router_generator`: crea router FastAPI con CRUD completo y middlewares condicionales.
3. `tests_generator`: diseña suite Pytest (unit, integration, performance básico, edge cases).
4. `infra_generator`: con `with_structured_output` para garantizar campos (`dockerfile`, `migration`, etc.).

Diseño de prompts: cada uno declara explícitamente criterios de calidad (ej. "nivel PRODUCCIÓN", "validaciones específicas", "logging estructurado"). Esto reduce alucinaciones.


### 4.4 Orquestación LCEL
Función `create_advanced_crud_pipeline()`:
- Fase base paralela: genera `modelo` y pasa `config` intacta.
- Lambda intermedia `_generate_dependent_components`: usa salida anterior para generar router y luego en paralelo tests + infraestructura.
- Se empaqueta todo en un `GeneratedComponents` final.

Ventaja: minimiza latencia (paraleliza lo que no depende) y mantiene orden lógico de dependencias (modelo → router → tests/infra).

### 4.5 Ejecución + Métricas
Se prepara input a partir de `sample_config` y se invoca `crud_pipeline.invoke(pipeline_input)`.

Métricas recolectadas manualmente (se esboza callback pero no se conecta en la ejecución actual):
- Tiempo total de generación
- Conteo de líneas por componente
- Número de clases, endpoints, validators, fixtures, asserts
- Cálculo de ROI: (tiempo manual estimado / tiempo IA)


### 4.6 Persistencia de Artefactos
Escribe a carpeta `generated_product_api/` solo archivos con contenido.
- `models.py`
- `routes.py`
- `test_api.py`
- `Dockerfile`
- `migration.py`


Si venías de:
- Prompt Engineering básico → ahora formalizas prompts como componentes reutilizables.
- FastAPI manual → ahora generas scaffolding consistente.
- DevOps / Infra → introduces IaC-like patterns para artefactos de backend.

Idea principal para recordar: **"Estandariza y paraleliza lo generable; reserva tu tiempo humano para lo verdaderamente diferencial."**

Checklist mental (READY):
- ¿Tengo config declarativa? ✅
- ¿Prompts con criterios explícitos? ✅
- ¿Control de dependencias y orden? ✅
- ¿Métricas de eficiencia? ✅
- ¿Artefactos persistidos y reutilizables? ✅

Si venías de:
- Prompt Engineering básico → ahora formalizas prompts como componentes reutilizables.
- FastAPI manual → ahora generas scaffolding consistente.
- DevOps / Infra → introduces IaC-like patterns para artefactos de backend.

Idea principal para recordar: **"Estandariza y paraleliza lo generable; reserva tu tiempo humano para lo verdaderamente diferencial."**

Checklist mental (READY):
- ¿Tengo config declarativa? ✅
- ¿Prompts con criterios explícitos? ✅
- ¿Control de dependencias y orden? ✅
- ¿Métricas de eficiencia? ✅
- ¿Artefactos persistidos y reutilizables? ✅